In [ ]:
REPO = "https://github.com/HenryNVP/anime_recomender.git"
!git clone $REPO

import os
%cd anime_recomender/

!pip -q install -r requirements.txt

import torch
print("CUDA available:", torch.cuda.is_available())

In [ ]:
import sys, shutil

PROJECT_ROOT = os.getcwd()
DATA_RAW   = os.path.join(PROJECT_ROOT, "data/raw")
os.makedirs(p, exist_ok=True)

from google.colab import files
uploaded = files.upload()   # select anime.csv and rating.csv 

shutil.move("anime.csv", "data/raw/anime.csv")
shutil.move("rating.csv", "data/raw/rating.csv")

## Data Preprocessing

In [ ]:
!python scripts/preprocess_data.py --data_dir data/raw --out_dir data/processed --build_item_features --check

: 

## Baseline Item-based Collaborative Filtering

In [ ]:
# Train baseline
!python -m src.baselines.itemcf.train --data_dir data/processed --out_prefix runs/itemcf/anime --k 100 --shrink 50

In [ ]:
# Evaluate baseline
!python -m src.baselines.itemcf.eval --model_prefix runs/itemcf/anime \
  --splits_dir data/processed/splits --split test --k 10,20

## Neural Matrix Factorization (NeuMF)

In [ ]:
# Train NeuMF
!python -m src.train --config configs/config_neumf.yaml

In [ ]:
# Evaluate NeuMF
!python -m src.eval --ckpt runs/neumf/[latest_run]/best.ckpt --config configs/config_neumf.yaml

## Matrix Factorization (Optional)

In [ ]:
# Train MF
!python -m src.train --config configs/config_mf.yaml

In [ ]:
# Evaluate MF
!python -m src.eval --ckpt runs/neumf/[latest_run]/best.ckpt --config configs/config_mf.yaml